In [1]:
import configparser
import pandas as pd
from utils.geocode import address_to_coords
from utils.filtering_for_addr import AddressFilter
from utils.apis import get_hospitals_by_condition, get_real_time_bed_info
from utils.direction import calculate_travel_time_and_sort

def main(address, conditions_korean=[]):
    """
    병원 데이터 처리 메인 함수
    :param address: 사용자 입력 주소 (도로명 주소 형식)
    :param conditions: 병원 필터링 조건 목록 (예: ["MKioskTy8", "MKioskTy10"])
    """
    # 설정 파일 로드
    config = configparser.ConfigParser()
    config.read('C:/Users/user/Desktop/24-2/졸업프로젝트/project_ai/keys.config')

    # AddressFilter 인스턴스 생성
    address_filter = AddressFilter('C:/Users/user/Desktop/24-2/졸업프로젝트/project_ai/keys.config')

    # 1. 사용자 주소 -> 좌표 변환
    api_key = config['API_KEYS']['kakao_api_key']
    user_coords = address_to_coords(address, api_key)

    if "error" in user_coords:
        print(f"오류: {user_coords['error']}")
        return

    user_lat = user_coords["lat"]
    user_lon = user_coords["lon"]
    print(f"사용자 좌표: 위도 {user_lat}, 경도 {user_lon}")

    # 2. 병원 조건 설정
    stage1, stage2 = address.split()[:2]  # Stage1 = 시도, Stage2 = 시군구

    # 3. 병원 조건에 맞는 hpid 수집
    condition_mapping = {
        "조산산모": "MKioskTy8",
        "정신질환자": "MKioskTy9",
        "신생아": "MKioskTy10",
        "중증화상": "MKioskTy11"
    }
    # 조건이 없으면 모든 병원을 검색
    if not conditions_korean:
        print("조건이 비어 있습니다. 모든 병원을 검색합니다.")
        conditions = []  # 빈 리스트로 설정
    else:
        conditions = [condition_mapping[cond] for cond in conditions_korean if cond in condition_mapping]

    hpid_list = get_hospitals_by_condition(stage1, stage2, conditions)
    if not hpid_list:
        print("조건에 맞는 병원이 없습니다.")
        return

    print(f"필터링된 병원 목록: {hpid_list}")

    # 4. 실시간 병상 정보 조회
    real_time_data = get_real_time_bed_info(stage1, stage2, hpid_list)
    if not real_time_data:
        print("실시간 병상 정보가 없습니다.")
        return

    # 5. 병상 정보 DataFrame 생성
    df = pd.DataFrame(real_time_data)

    # 6. enriched_df 생성 및 저장
    enriched_df = address_filter.enrich_filtered_df(df)

    # 7. 소요 시간 계산 및 정렬
    enriched_df = calculate_travel_time_and_sort(enriched_df, user_lat, user_lon)

    # 추가된 칼럼 확인
    if "distance_km" in enriched_df.columns and "travel_time_h" in enriched_df.columns:
        print("거리와 소요 시간이 추가된 데이터:")
        print(enriched_df[["distance_km", "travel_time_h", "travel_time_m", "travel_time_s"]].head())

    # 8. 결과 저장
    enriched_df.to_csv("enriched_filtered_df.csv", index=False, encoding="utf-8-sig")
    print("결과가 'enriched_filtered_df.csv' 파일로 저장되었습니다.")
    
    return enriched_df


In [2]:
if __name__ == "__main__":
    #조산산모(MKioskTy8), 정신질환자(MKioskTy9), 신생아(MKioskTy10), 중증화상(MKioskTy11)
    enriched_df = main(address="서울 중구 을지로 245")#, 
                       #conditions_korean= ["정신질환자"] )
    if enriched_df is not None:
        print(enriched_df.head(5))

사용자 좌표: 위도 37.5677264605676, 경도 127.005484601979
조건이 비어 있습니다. 모든 병원을 검색합니다.
생성된 Redis 키: hospitals:서울:중구:all
저장된 hpid Redis 캐시 데이터: ["A1100052"]
Redis 캐시에서 병원 데이터 로드
필터링된 병원 목록: ['A1100052']
생성된 Redis 키: real_time_bed_info:A1100052
저장된 응급실 Redis 캐시 데이터: {"dutyName": "\uad6d\ub9bd\uc911\uc559\uc758\ub8cc\uc6d0", "dutyTel3": "02-2276-2114", "hpid": "A1100052", "hv10": "N", "hv11": "Y", "hv22": "0", "hv25": "0", "hv26": "0", "hv27": "5", "hv28": "3", "hv29": "2", "hv3": "0", "hv30": "1", "hv31": "2", "hv35": "2", "hv36": "0", "hv38": "11", "hv39": "2", "hv41": "6", "hv42": "Y", "hv5": "Y", "hv7": "Y", "hv9": "12", "hvamyn": "Y", "hvangioayn": "Y", "hvcrrtayn": "Y", "hvctayn": "Y", "hvec": "12", "hvecmoayn": "Y", "hvgc": "84", "hvhypoayn": "Y", "hvicc": "8", "hvidate": "20241231154906", "hvincuayn": "Y", "hvmriayn": "Y", "hvoc": "5", "hvoxyayn": "N1", "hvs01": "15", "hvs02": "3", "hvs03": "2", "hvs04": "3", "hvs05": "6", "hvs07": "0", "hvs14": "20", "hvs17": "13", "hvs18": "2", "hvs19": "0

In [3]:
enriched_df.columns

Index(['dutyName', 'dutyAddr', 'wgs84Lat', 'wgs84Lon', 'is_trauma', 'dutyTel3',
       'hpid', 'hv10', 'hv11', 'hv22', 'hv25', 'hv26', 'hv27', 'hv28', 'hv29',
       'hv3', 'hv30', 'hv31', 'hv35', 'hv36', 'hv38', 'hv39', 'hv41', 'hv42',
       'hv5', 'hv7', 'hv9', 'hvamyn', 'hvangioayn', 'hvcrrtayn', 'hvctayn',
       'hvec', 'hvecmoayn', 'hvgc', 'hvhypoayn', 'hvicc', 'hvidate',
       'hvincuayn', 'hvmriayn', 'hvoc', 'hvoxyayn', 'hvs01', 'hvs02', 'hvs03',
       'hvs04', 'hvs05', 'hvs07', 'hvs14', 'hvs17', 'hvs18', 'hvs19', 'hvs21',
       'hvs22', 'hvs23', 'hvs25', 'hvs26', 'hvs27', 'hvs28', 'hvs29', 'hvs30',
       'hvs31', 'hvs32', 'hvs33', 'hvs34', 'hvs35', 'hvs38', 'hvs54', 'hvs57',
       'hvs58', 'hvs59', 'hvventiayn', 'hvventisoayn', 'phpid', 'rnum',
       'travelTime', 'distance_km', 'congestion', 'travel_time_h',
       'travel_time_m', 'travel_time_s'],
      dtype='object')

In [5]:
enriched_df[['dutyName','dutyAddr','dutyTel3', "distance_km",
             "travel_time_h", "travel_time_m", "travel_time_s", 'congestion','hvamyn', 'is_trauma']]

,dutyName,dutyAddr,dutyTel3,distance_km,travel_time_h,travel_time_m,travel_time_s,congestion,hvamyn,is_trauma
0,국립중앙의료원,"서울특별시 중구 을지로 245, 국립중앙의료원 (을지로6가)",02-2276-2114,0.149,0,0,44,알 수 없음,Y,True
